In [4]:
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import  make_subplots
from os import path

In [29]:
def snap(fig, filename, figuredir = '../figures') :
    fig.update_layout(width=700, height=300,
                      margin = dict(l=1, r=1, t=20, b=20))
    try :
        fig.write_image(path.join(figuredir, filename))
    except:
        pio.orca.shutdown_server()
        fig.write_image(path.join(figuredir, filename))
    return fig

In [3]:
import pandas as pd
import numpy as np
import pickle
from glob import glob
from os.path import basename, join

from tqdm.notebook import tqdm

from sklearn.model_selection import LeaveOneGroupOut, cross_validate, learning_curve, cross_val_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [8]:
def get_data(r,k,data_prefix) :
    filename  = f'{data_prefix}/r{r}_{k}mer.pkl'
    data = pd.read_pickle(filename)
    data = data[ (data.methylatedin == 0) | (data.methylated_ratio >0.75)]

    X = data.iloc[:,:data.attrs['n_kmers']]
    y = (data.methylatedin>1) & (data.methylated_ratio>0.75)
    seqid = data.index.get_level_values('seqid')
    
    filename = '/'.join(filename.split('/')[-2:])

    print (f'{filename}: size: {len(y)}, pos: {y.sum()}, baseline: {y.sum()/len(y):.2f}')
    
    return X,y,seqid

def get_model(r,k,model_prefix) :
    with open(f'{model_prefix}/r{r}_{k}mer_logistic.pkl', 'rb') as f :
        return pickle.load(f)
    
def pr_auc(model, X_test, y_test) :
    y_pred = model.predict_proba(X_test)[:,1]
    precision, recall, threshold = precision_recall_curve(y_test, y_pred)
    return auc(recall, precision)

In [6]:
%run ../scripts/utils.py

# Exome models

We import some pre-trained models (exome) to see how they did.

In [13]:
model_prefix = join('../models','Apis_mellifera', 'exome')
data_prefix = join('../data','Apis_mellifera', 'exome')

In [14]:
models=dict(L1=dict(), L2=dict())
mean_scores = dict(L1=dict(), L2=dict())
std_scores = dict(L1=dict(), L2=dict())

kmers = dict()


for file in glob(f'{model_prefix}/*_logistic.pkl') :
    r, k, tmp = basename(file).split('_')
    
    r = int(r[1:])
    k = int(k[0])

    if r not in models['L1'] :
        models['L1'][r] = dict()
        models['L2'][r] = dict()

        mean_scores['L1'][r] =dict()
        mean_scores['L2'][r] =dict()

        std_scores['L1'][r] =dict()
        std_scores['L2'][r] =dict()

        
    with open(file, 'rb') as f :
        model = pickle.load(f)
        
        models['L1'][r][k] = model['l1']['estimator']
        mean_scores['L1'][r][k] = model['l1']['test_score'].mean()
        std_scores['L1'][r][k] = model['l1']['test_score'].std()
        
        models['L2'][r][k] = model['l2']['estimator']
        mean_scores['L2'][r][k] = model['l2']['test_score'].mean()
        std_scores['L2'][r][k] = model['l2']['test_score'].std()
        
        kmers[k] = model['features']
        

mean_scores_L1 = pd.DataFrame(mean_scores['L1']).round(3)
mean_scores_L1.sort_index(axis=0, inplace=True)
mean_scores_L1.sort_index(axis=1, inplace=True)

mean_scores_L2 = pd.DataFrame(mean_scores['L2']).round(3)
mean_scores_L2.sort_index(axis=0, inplace=True)
mean_scores_L2.sort_index(axis=1, inplace=True)

In [30]:
#std_scores = pd.DataFrame(std_scores).round(3)
#std_scores.sort_index(axis=0, inplace=True)
#std_scores.sort_index(axis=1, inplace=True)

#mean_scores.drop(100, axis=1, inplace=True)
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=('L1', 'L2'))
fig.add_trace(go.Heatmap(z=mean_scores_L1, y = mean_scores_L1.index, x = mean_scores_L1.columns, coloraxis="coloraxis"), row=1, col=1)
fig.add_trace(go.Heatmap(z=mean_scores_L2, y = mean_scores_L2.index, x = mean_scores_L2.columns, coloraxis="coloraxis"), row=1,col=2)

fig.update_xaxes(title_text="radius", row=1, col=1, tickmode = 'array', tickvals = mean_scores_L1.columns)
fig.update_xaxes(title_text="radius", row=1, col=2, tickmode = 'array', tickvals = mean_scores_L1.columns)
fig.update_yaxes(title_text="k", tickmode = 'array', tickvals = mean_scores_L1.index)

fig.show()

#snap(fig, 'logistic_grid.pdf')

We note that L1 actually did better. 
Due to overlap of words, therefore correlation of reatures, this is not a huge surprise. 

# Testing on whole genome

We test the models that were trained on exons on all cpgs. 

In [16]:
data_prefix = join('../data','Apis_mellifera', 'genome')

mean_scores['genome'] = dict()

for file in tqdm(glob(f'{data_prefix}/r*_*mer.pkl')) :
    r, k = basename(file).split('_')
    
    r = int(r[1:])
    k = int(k[0])
    
    if r not in mean_scores['genome'] :
        mean_scores['genome'][r]=dict()
    
    X,y, seqid = get_data(r,k,data_prefix)
    
    scores = list()
    
    for model, (train, test) in zip(models['L1'][r][k], LeaveOneGroupOut().split(X,y,seqid)) :

        scores.append (pr_auc(model, X.take(test),  y.take(test)))

    mean_scores['genome'][r][k] = np.array(scores).mean()

genome/r300_5mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r200_5mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r200_6mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r400_6mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r100_6mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r150_6mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r500_6mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r500_5mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r250_6mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r250_5mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r300_4mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r450_4mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r400_4mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r350_4mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r300_6mer.pkl: size: 1528746, pos: 35296, baseline: 0.02
genome/r450_5mer.pkl: size: 1528746, pos

In [17]:
mean_scores_genome = pd.DataFrame(mean_scores['genome']).round(3)
mean_scores_genome.sort_index(axis=0, inplace=True)
mean_scores_genome.sort_index(axis=1, inplace=True)

In [97]:
fig = make_subplots(rows=1, cols=2, 
                    shared_yaxes=True,
                   # horizontal_spacing = 0.025,
                    subplot_titles=('Test on exome', 'Test on genome', 'Retained in genome'))

tmp = mean_scores_L1.reindex_like(mean_scores_genome)
fig.add_trace(go.Heatmap(z=tmp, 
                         y = tmp.index, 
                         x = tmp.columns, 
                         coloraxis="coloraxis"), row=1, col=1)

fig.add_trace(go.Heatmap(z=mean_scores_genome, 
                         y=mean_scores_genome.index, 
                         x=mean_scores_genome.columns, 
                         coloraxis="coloraxis"), row=1,col=2)

#tmp = mean_scores_genome/mean_scores_L1.reindex_like(mean_scores_genome)
#fig.add_trace(go.Heatmap(z=tmp, 
#                         y = tmp.index, 
#                         x = tmp.columns, 
#                         coloraxis="coloraxis"), row=1, col=3)


fig.update_xaxes(title_text="Radius (r)",tickmode = 'array', tickvals = mean_scores_L1.columns)
fig.update_yaxes(title_text="k", tickmode = 'array', tickvals = mean_scores_L1.index, row=1, col=1)
fig.update_layout(coloraxis_colorbar_thickness=20, coloraxis_colorbar_title='AUC')

snap(fig, 'logistic_exon-genome_grid.pdf')

# Training on entire genome

We observe that r=500 and k=5 retains the most performance when generalizing from exome to whole genome. 
86.3% of the AUC (0.855) on exome is retained to yield an AUC of 0.738 on the whole genome.

Now we re-fit the model to the whole genome data to see whether it improves. 

Note that this takes a long time and actually reduces the auc to 0.727

In [500]:
organism='Apis_mellifera'
data_prefix = join('../data',organism, 'genome')

k=5
r=500

#retain large dataset so we don't have to keep loading it
X_genome, y_genome, seqid_genome = get_data(r,k,data_prefix) 

model = models['L1'][r][k][0]
model['logisticregression'].set_params(warm_start = True)

genome_scores = cross_validate(model, X_genome , 
                               y_genome, 
                               cv=LeaveOneGroupOut().split(X_genome,y_genome,seqid_genome),
                               scoring=pr_auc, 
                               n_jobs=16,
                               return_estimator=True,
                               pre_dispatch = 'n_jobs',
                               verbose = 2)

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  10 out of  16 | elapsed: 105.2min remaining: 63.1min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed: 120.3min finished


In [502]:
genome_scores['test_score'].mean()

0.7271409393054988

# Tuning regularization strength

We choose the best model and increase regularization strength until it breaks

In [54]:
data_prefix = '../data/Apis_mellifera/exome'
model_prefix ='../models/Apis_mellifera/exome'
r, k = 500, 5

model = get_model(r,k,model_prefix)['l1']['estimator'][0]

X, y, seqid = get_data(r, k, data_prefix)

c_scores = dict()

for C in [0.01, 0.005, 0.001, 0.0005, 0.0001] :

    model['logisticregression'].set_params(penalty='l1', C=C, warm_start=True)

    scores = cross_validate(model, X, y, cv=LeaveOneGroupOut().split(X,y,seqid),
                            scoring=pr_auc, 
                            n_jobs=16,
                            return_estimator=True,
                            verbose = 1)
    c_scores[C] = scores

exome/r500_5mer.pkl: size: 179042, pos: 28051, baseline: 0.16


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   48.3s remaining:  5.6min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:  1.3min finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   34.2s remaining:  4.0min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   57.5s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   15.0s remaining:  1.8min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   18.6s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   12.8s remaining:  1.5min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   15.2s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parall

In [55]:
for c,scores in c_scores.items() :
    print (c, scores['test_score'].mean().round(2), scores['test_score'].std().round(2))

0.01 0.85 0.03
0.005 0.86 0.03
0.001 0.85 0.02
0.0005 0.85 0.02
0.0001 0.76 0.03


# Learning Curve

In [52]:
train_frac = np.linspace(0.1, 1.0, 10)

train_scores=dict()
test_scores=dict()

for C in [0.01,0.001] :

    model['logisticregression'].set_params(warm_start=False, C=C)

    train_sizes, train_scores[C], test_scores[C] = learning_curve(model, X, y, groups=seqid, 
                        cv=LeaveOneGroupOut().split(X,y,seqid), 
                        scoring=pr_auc, 
                        n_jobs = 16 , pre_dispatch='n_jobs',
                        verbose = 2,
                        train_sizes = train_frac)

[learning_curve] Training set sizes: [ 15716  31432  47148  62864  78581  94297 110013 125729 141445 157162]


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  25 tasks      | elapsed:   39.2s
[Parallel(n_jobs=16)]: Done 160 out of 160 | elapsed:  4.0min finished


[learning_curve] Training set sizes: [ 15716  31432  47148  62864  78581  94297 110013 125729 141445 157162]


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  25 tasks      | elapsed:   13.7s
[Parallel(n_jobs=16)]: Done 160 out of 160 | elapsed:  1.4min finished


In [83]:
fig = make_subplots(rows=1, cols=2, 
                    shared_yaxes=True, 
                    subplot_titles=['Moderate regularization', 'Strong regularization'])

test_marker=dict(color='red', opacity=1) 
train_marker = dict(color='green', opacity=1) 

fig.add_traces([ go.Scatter(x=train_frac, 
                       y=train_scores[0.01].mean(axis=1), 
                       error_y=dict( type='data', array=train_scores[0.01].std(axis=1),visible=True),
                       name='train', marker=train_marker, legendgroup = 'all'),
            go.Scatter(x=train_frac, 
                      y= test_scores[0.01].mean(axis=1), 
                      error_y=dict( type='data', array=test_scores[0.01].std(axis=1),visible=True),
                      name='test', marker=test_marker, legendgroup='all')], rows=(1,1), cols=(1,1))

fig.add_traces( [go.Scatter(x=train_frac, 
                       y=train_scores[0.001].mean(axis=1), 
                       error_y=dict( type='data', array=train_scores[0.001].std(axis=1),visible=True),
                       name='train', marker=train_marker, legendgroup = 'all', showlegend=False),
                 go.Scatter(x=train_frac, 
                      y= test_scores[0.001].mean(axis=1), 
                      error_y=dict( type='data', array=test_scores[0.001].std(axis=1),visible=True),
                      name='test', marker=test_marker, legendgroup='all', showlegend=False)], rows=(1,1), cols=(2,2))

fig.update_layout(legend=dict(x=0.3, y=0.15))

fig.update_xaxes(title = 'Training set size')

fig.update_yaxes(title = 'Score', row=1, col=1)

snap(fig, 'learning_curve.pdf')

# Coefficients

In [56]:
def get_coefs(scores, features) :
    coefs = []

    for model in scores['estimator'] :
        coefs.append(model['logisticregression'].coef_.flatten())

    return pd.DataFrame(np.array(coefs), columns=features ).T

def get_top(coefs, n) :
    top = set()
    for i in coefs : 
        for kmer in coefs.abs().sort_values(i).head(n).index :
            top.add(kmer)
            
    return top

In [79]:
fig = make_subplots(rows=1, cols=2)

coefs = get_coefs(c_scores[0.01], kmers[k])

mean_coefs = coefs.mean(axis=1).sort_values(ascending=False)
std_coefs = coefs.std(axis=1).reindex(mean_coefs.index)

weak_coefs = go.Scatter(name = 'moderate regularization' ,
                        y=mean_coefs, 
                        error_y=dict(type='data', array=std_coefs, visible=True) )

coefs = get_coefs(c_scores[0.001], kmers[k])

mean_coefs = coefs.mean(axis=1).sort_values(ascending=False)
std_coefs = coefs.std(axis=1).reindex(mean_coefs.index)

strong_coefs = go.Scatter(name= 'strong regularization', 
                          y= mean_coefs, 
                          error_y=dict(type='data', array=std_coefs, visible=True) )

fig.add_traces([weak_coefs, strong_coefs], rows=(1,1), cols=(1,1))


top_n = dict()
for i in range(1,50) :
    top = get_top(coefs, i)
    top_n[i]=len(top)
top_n = pd.Series(top_n)

fig.add_traces([ go.Scatter(x=top_n.index, y=top_n, showlegend=False), 
                 go.Scatter(x=(1,25,50), y=(1,25,50), showlegend=False,
                            text = ['','x=y',''],
                            textposition='bottom right',
                            mode='lines+text', line_dash='dash')], rows=(1,1), cols=(2,2) )

fig.update_layout(legend=dict(x=0.05, y=0.15),
                  width=800, height=400,
                  margin = dict(l=1, r=1, t=20, b=20))

fig.update_xaxes(title = 'Pentamers', row=1, col=1)
fig.update_xaxes(title = 'n', row=1, col=2)

fig.update_yaxes(title = 'Coefficient', row=1, col=1)
fig.update_yaxes(title = 'Pentamers in top n across folds', title_standoff=5, row=1, col=2)

snap(fig, 'feature_selection.pdf')

# Select nonzero features

With the way tanking looks, no amount of statistical testing will do much. 
Crude guess as good as any, so we chose those coefs 2 std away from 0.

In [71]:
nonzero_coefs = coefs[(coefs.mean(axis=1).abs()>2*coefs.std(axis=1))]
nonzero = nonzero_coefs.index.to_list()
len(nonzero)

97

In [72]:
positive = nonzero_coefs[nonzero_coefs.mean(axis=1)>0].index
negative = nonzero_coefs[nonzero_coefs.mean(axis=1)<0].index
print(f'{len(positive)} enhancing and {len(negative)} repressing kmers.')

50 enhancing and 47 repressing kmers.


# Clustering selected kmers

In [19]:
from scipy.spatial.distance import hamming
from sklearn.cluster import AgglomerativeClustering

In [952]:
def cluster_kmers(kmers, threshold=0.21) :    
    tmp = list(map(list, kmers))

    dist = np.zeros((len(tmp), len(tmp)))

    for i, kmer1 in enumerate(tmp) :
        for j, kmer2 in enumerate(tmp[i+1:], start=i+1) :
            dist[i,j] = dist[j,i]= hamming(kmer1, kmer2)

    dist =  pd.DataFrame(dist, index=kmers, columns=kmers )

    agg = AgglomerativeClustering(affinity='precomputed', 
                                  linkage='complete', 
                                  distance_threshold=threshold, 
                                  n_clusters=None)

    clusters = pd.Series(agg.fit_predict(dist), index=kmers)

    return clusters    

In [928]:
rc_positive = [str(Seq.Seq(kmer).reverse_complement()) for kmer in positive]

In [935]:
full_positive = positive.to_list()+rc_positive

In [911]:
tmp = list(map(list, negative))

dist = np.zeros((len(tmp), len(tmp)))

for i, kmer1 in enumerate(tmp) :
    for j, kmer2 in enumerate(tmp[i+1:], start=i+1) :
        dist[i,j] = dist[j,i]= hamming(kmer1, kmer2)
        
dist =  pd.DataFrame(dist, index=negative, columns=negative )

agg = AgglomerativeClustering(affinity='precomputed', linkage='complete', distance_threshold=0.21, n_clusters=None)

clusters = pd.Series(agg.fit_predict(dist), index=negative)

clusters.value_counts().value_counts()

1    10
2     8
4     3
3     3
dtype: int64

# Position distribution of selected kmers

In [90]:
from Bio import Seq, SeqIO

In [91]:
rc = { kmer:str(Seq.Seq(kmer).reverse_complement()) for kmer in nonzero}

In [94]:
sequences = list(SeqIO.parse(f'../data/Apis_mellifera/exome/r{r}.fa', 'fasta'))

In [95]:
robust_counts = np.zeros((len(nonzero), 2*r+2))
never_counts = np.zeros((len(nonzero), 2*r+2))

for record in tqdm(sequences, leave=False) :
    pos, data = parse_fasta_description(record)
    seq = record.seq.upper()

    for i, kmer in enumerate(nonzero) : 
        
        j=0
        while j<len(seq):
            j = seq.find(kmer, j+1)
            
            if j>= 0 :
                if data['methylatedin'] >1 and data['methylated_ratio'] >0.75 :
                    
                    robust_counts[i,j:j+k] +=1
                
                elif data['methylatedin'] == 0:
                    never_counts[i,j:j+k] +=1
            else : 
                break
        
        while j<len(seq):
            j = seq.find(rc[kmer], j+1)
            
            if j>= 0 :
                if data['methylatedin'] >1 and data['methylated_ratio'] >0.75 :
                    
                    robust_counts[i,j:j+k] +=1
                
                elif data['methylatedin'] == 0:
                    never_counts[i,j:j+k] +=1
            else : 
                break

In [101]:
radial_pos = np.concatenate( [np.arange(-r, 1), np.arange(0, r+1)])
block_size = 20


df_robust = pd.DataFrame(robust_counts, index=nonzero, columns = radial_pos).T
# Fold over symmetry
df_robust = df_robust.groupby(df_robust.index.to_series().abs()).sum()
# Trim
df_robust = df_robust.loc[1:]
# Block
df_robust = df_robust.groupby(df_robust.index // block_size).sum()
# Normalize
df_robust /= df_robust.sum()

df_never = pd.DataFrame(never_counts, index=nonzero, columns = radial_pos ).T
# Fold over symmetry
df_never = df_never.groupby(df_never.index.to_series().abs()).sum()
# Trim
df_never = df_never.loc[1:]
# Block
df_never = df_never.groupby(df_never.index // block_size).sum()
# Normalize
df_never /= df_never.sum()

# Odds ratio
df=df_robust/df_never
#Trim
df = df.iloc[:-1]

In [103]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

pos_trace = go.Heatmap(z=df[positive] , x=df[positive].columns, y=df[positive].index*block_size)

neg_trace = go.Heatmap(z=df[negative] , x=df[negative].columns, y=df[negative].index*block_size)

fig.add_traces([pos_trace, neg_trace], rows=(1,1), cols=(1,2))
fig.update_traces( zmin=0.5, zmax=1.5)

In [108]:
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list, optimal_leaf_ordering

from scipy.spatial.distance import pdist

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.025, 
                    shared_yaxes=True)

tmp = df[positive].T
metric='euclidean'
method='ward'

Z = linkage(tmp, method=method, metric=metric, optimal_ordering=True)

tmp2 = tmp.iloc[leaves_list(Z)].T

fig.add_trace(go.Heatmap(z=tmp2, 
                         x=tmp2.columns, 
                         y=tmp2.index*block_size,  
                         coloraxis='coloraxis',
                         zmin=0.5, zmax=1.5), row=1, col=1)

tmp = df[negative].T

Z = linkage(tmp, method=method, metric=metric, optimal_ordering=True)

tmp2 = tmp.iloc[leaves_list(Z)].T

fig.add_trace(go.Heatmap(z=tmp2, 
                         x=tmp2.columns, 
                         y=tmp2.index*block_size,  
                         coloraxis='coloraxis',
                         zmin=0.5, zmax=1.5), row=1, col=2)

fig.update_layout(yaxis_title='Distance from CpG', 
                 xaxis_title='Enhancing kmers', 
                 xaxis2_title= 'Repressing kmers',
                 coloraxis_colorbar_title='Odds ratio')

fig.update_xaxes(showticklabels=False)
snap(fig, 'kmer_positions.pdf')

In [892]:
df = pd.DataFrame(dict(mean=nonzero_coefs[nonzero_coefs.mean(axis=1)>0].mean(axis=1), 
                       std=nonzero_coefs[nonzero_coefs.mean(axis=1)>0].std(axis=1)))
df.sort_values('mean',ascending=False, inplace=True)

df = df.round(3)

middle = int(np.ceil(len(df)/2))

df1 = df.iloc[:middle].reset_index()
df2 = df.iloc[middle:].reset_index()

df1.join(df2, how='outer', rsuffix= '2').to_latex('enhancing.tex', caption='Enhancing kmers and associated coefficients')

In [884]:
df = pd.DataFrame(dict(mean=nonzero_coefs[nonzero_coefs.mean(axis=1)<0].mean(axis=1), 
                       std=nonzero_coefs[nonzero_coefs.mean(axis=1)<0].std(axis=1)))
df.sort_values('mean',ascending=True, inplace=True)

df = df.round(3)

middle = int(np.ceil(len(df)/2))

df1 = df.iloc[:middle].reset_index()
df2 = df.iloc[middle:].reset_index()

df1.join(df2, how='outer', rsuffix= '2').to_latex('repressing.tex', caption='Repressing kmers and associated coefficients')

# Precision-recall curve

In [127]:
r,k = 500,5

model = c_scores[0.001]
t = np.linspace(0, 1, 1000)

X,y, seqid = get_data(r,k,'../data/Apis_mellifera/exome')

precisions, recalls = [],[]
for model, (train, test) in zip(models['L1'][r][k], LeaveOneGroupOut().split(X,y,seqid)) :

    X_test = X.take(test)
    y_test = y.take(test)

    y_pred = model.predict_proba(X_test)[:,1]

    precision, recall, threshold = precision_recall_curve(y_test, y_pred)

    interp_precision = np.interp(t, threshold, precision[:-1])
    interp_recall = np.interp(t, threshold, recall[:-1])

    recalls.append(interp_recall)
    precisions.append(interp_precision)
    
precision_mean_exome  = np.array(precisions).mean(axis=0)
precision_std_exome = np.array(precisions).std(axis=0)
recall_mean_exome  = np.array(recalls).mean(axis=0)
recall_std_exome = np.array(recalls).std(axis=0)


X,y, seqid = get_data(r,k,'../data/Apis_mellifera/genome')

precisions, recalls = [],[]
for model, (train, test) in zip(models['L1'][r][k], LeaveOneGroupOut().split(X,y,seqid)) :

    X_test = X.take(test)
    y_test = y.take(test)

    y_pred = model.predict_proba(X_test)[:,1]

    precision, recall, threshold = precision_recall_curve(y_test, y_pred)

    interp_precision = np.interp(t, threshold, precision[:-1])
    interp_recall = np.interp(t, threshold, recall[:-1])

    recalls.append(interp_recall)
    precisions.append(interp_precision)
    
precision_mean_genome  = np.array(precisions).mean(axis=0)
precision_std_genome = np.array(precisions).std(axis=0)
recall_mean_genome  = np.array(recalls).mean(axis=0)
recall_std_genome = np.array(recalls).std(axis=0)


exome/r500_5mer.pkl: size: 179042, pos: 28051, baseline: 0.16
genome/r500_5mer.pkl: size: 1528746, pos: 35296, baseline: 0.02


In [134]:
trace1= go.Scatter(x=recall_mean_exome, 
                   y=precision_mean_exome, 
                   error_x=dict(array=recall_std_exome, type='data'), 
                   error_y=dict(array=precision_std_exome, type='data'),
                  name ='exome')

trace2= go.Scatter(x=recall_mean_genome, 
                   y=precision_mean_genome, 
                   error_x=dict(array=recall_std_genome, type='data'), 
                   error_y=dict(array=precision_std_genome, type='data'),
                   name = 'genome')

go.Figure([trace1, trace2])

logistic_pr=dict(exome=trace1, genome=trace2)

with open('logistic_pr.pkl', 'wb') as f :
    pickle.dump(dict(exome=trace1, genome=trace2), f)